In [2]:
import pandas as pd
import re
import sys, getopt
import jsonlines
import random
import numpy as np
from tqdm import tqdm
import pandas as pd
import re
import numpy as np
import os
import time
from openai import OpenAI
import pandas as pd
import jsonlines
import numpy as np
from tqdm import tqdm
import json
import tiktoken
import ollama
encoding = tiktoken.encoding_for_model("gpt-4")
import os
import sys
import logging
from tqdm import tqdm
import csv 

# CHAGNE PROMPT HEREEEEEEE
with open('../prompts/prompt1-10.txt', 'r') as file:
    sys_prompt = file.read()

modelfile=f'''
FROM llama3.3
system """
{sys_prompt.strip()}
"""
'''
model = 'myexample1'
num_ctx = 25000 # 15000 original,  - works with 17000 with old GPUs 
ollama.create(model='myexample1', modelfile=modelfile)



{'status': 'success'}

In [3]:
input_folder = "./input"
csv_dir = input_folder + "/Execution-Paths/"
access_control_file = "/u1/hfaheem/DLAndroidArtifact/my-paths-code/input/Entrypoints.csv"
output_folder = "./output/week1/output1"


In [4]:
def process_csv_files(csv_directory):
    """
    Process all CSV files in the specified directory and structure the data into a DataFrame.
    
    Args:
        csv_directory (str): Path to directory containing CSV files
        
    Returns:
        pd.DataFrame: Structured data with service_name, class, method, and depths
    """
    data_dict = {}
    
    # Process each CSV file
    for filename in os.listdir(csv_directory):
        if filename.endswith('.csv'):
            service_name = os.path.splitext(filename)[0]
            filepath = os.path.join(csv_directory, filename)
            
            with open(filepath, 'r', newline='', encoding='utf-8') as csvfile:
                reader = csv.DictReader(csvfile)
                for row in reader:
                    try:
                        class_name = row['Class'].strip('"')
                        method = row['Method'].strip('"')
                        depth = row['Depth'].strip('"')
                        trace = row['Trace Instruction(s) ...'].strip('"')
                        java_code = row['Java Code Representation'].strip('"')
                        access_control_level = row['Access Control Level'].strip('"')
                        # if access_control_level == 'NONE':
                        #     print(f"found it: {filename}")
                        # Create unique key for grouping
                        key = (service_name, class_name, method, access_control_level)
                        
                        # Create depth entry
                        depth_entry = {
                            'depth': int(depth),
                            'trace': trace,
                            'java_code': java_code
                        }
                        
                        # Add to data dictionary
                        if key not in data_dict:
                            data_dict[key] = []
                        data_dict[key].append(depth_entry)
                        
                    except KeyError as e:
                        print(f"Missing column in {filename}: {e}")
                        continue
    
    # Convert to DataFrame
    rows = []
    for key in data_dict:
        service, cls, method, access_control_level = key
        depths = sorted(data_dict[key], key=lambda x: x['depth'])  # Sort by depth
        rows.append({
            'service_name': service,
            'class': cls,
            'method': method,
            'depths': depths,
            'access control level': access_control_level
        })
    
    return pd.DataFrame(rows)

def get_java_code(row):
# get the maximum depth
    max_depth = max([x['depth'] for x in row["depths"]])
    # get all the java code for the max_depth
    code_string = ""
    counter = 1
    for code in row["depths"]:
        if code['depth'] == max_depth:
            code_string += f"This is path {counter} for the API with depth {max_depth}:\n"
            code_string += code['java_code']
            counter += 1

    return code_string


df = process_csv_files(csv_dir)
# df.to_parquet("android_services_methods.parquet")

In [5]:
def run_first_prompt_Ollama(method_code,run):
    """ runs the first prompt - extract sinks from the traces
    """
    
    global sys_prompt

    # user_prompt = get_method_traces_from_file(file_path, interface, method)
    
    user_prompt = method_code
    if(not run):
        return {
            "system_message": sys_prompt,
            "user_message": user_prompt,
            "response": """
                    After analyzing the provided Java code for the `addInstrumentationResults` API, I have identified one potential sink. Here is the JSON output:

```
{
  "Sinks": [
    [
      "int var5 = UserHandle.getCallingUserId();",
      "ProcessRecord var16 = getRecordForAppLocked(param2);",
      "IApplicationThread var26 = var25.thread;",
      "IApplicationThread var27 = var25.thread;"
    ]
  ]
}
```

Ranking: This sink is ranked as the most sensitive.
    
            """
        }
        
        
    
    response = ollama.chat(model=model, messages=[
    {
        'role': 'user',
        'content': user_prompt,
    },
    ]
    ,
     options={
        'num_ctx': num_ctx,
        # 'temperature': 0.3 # Todo : CHECK TEMPERATURE HERERERERERERERER
    }
    
    )
    
    # logging.info(f"Response for {method} = {response['message']['content']}")

    
    return {
        "system_message": sys_prompt,
        "user_message": user_prompt,
        "response": response['message']['content']
    }
    

# get the json from the res["response"]

def extract_json_from_string(input_string):
    """
    Extracts JSON from the given string.

    Args:
        input_string (str): The string containing embedded JSON.

    Returns:
        dict: The extracted JSON as a Python dictionary.
    """
    try:
        # Use a regex pattern to extract the JSON part
        json_pattern = r"```(?:json)?\n(.*?)\n```"
        match = re.search(json_pattern, input_string, re.DOTALL)
        if match:
            json_string = match.group(1)
            return json.loads(json_string)
        else:
            raise ValueError("No JSON found in the provided string.")
    except json.JSONDecodeError as e:
        raise ValueError(f"Error decoding JSON: {e}")



In [6]:
df.columns

Index(['service_name', 'class', 'method', 'depths', 'access control level'], dtype='object')

In [7]:
test = df[df["method"] == "cancelVibrate(ILandroid/os/IBinder;)V"]["depths"]
first_element = test.iloc[0]
first_element

[{'depth': 0,
  'trace': '10 = getfield < Application, Lcom/android/server/vibrator/VibratorManagerService, mLock, <Application,Ljava/lang/Object> > 1,\nmonitorenter 10,\n11 = new <Application,Lcom/android/server/vibrator/Vibration$EndInfo>@20,\n12 = getstatic < Application, Lcom/android/server/vibrator/Vibration$Status, CANCELLED_BY_USER, <Application,Lcom/android/server/vibrator/Vibration$Status> >,\n16 = getfield < Application, Lcom/android/server/vibrator/VibratorManagerService, mNextVibration, <Application,Lcom/android/server/vibrator/VibrationStepConductor> > 1,\nconditional branch(eq, to iindex=-1) 16,17,\n19 = invokevirtual < Application, Lcom/android/server/vibrator/VibrationStepConductor, getVibration()Lcom/android/server/vibrator/Vibration; > 16 @35 exception:18,\n21 = invokevirtual < Application, Lcom/android/server/vibrator/VibratorManagerService, shouldCancelVibration(Lcom/android/server/vibrator/Vibration;ILandroid/os/IBinder;)Z > 1,19,2,3 @39 exception:20,\nconditional 

In [8]:
df["service_name"].value_counts()
df[df["service_name"] == "Lcom.android.server.pm.UserManagerService"]["access control level"].value_counts()
df_ums = df[df["service_name"] == "Lcom.android.server.pm.UserManagerService"]


In [9]:


counter = 0
df_ums['json_answer'] = None
for index, row in tqdm(df_ums.iterrows(), total=df_ums.shape[0], desc="Processing rows"):
    # print(row['method'].split("(")[0])
    # print(row['service_name'])
    method_name = row['method'].split("(")[0]
    service_name = row['service_name']
    code_string = get_java_code(row)
    
    res = run_first_prompt_Ollama(code_string,True)
    json_answer = None
    try:
        json_answer = extract_json_from_string(res["response"])
    except Exception as e:
        print(f"Error extracting JSON from response: {e}")
        print(method_name)
        print(service_name)
        # continue
        # Store the json_answer in the new column
    df_ums.at[index, 'json_answer'] = json_answer

    # Write response details to files
    folder_path = f'{output_folder}/{service_name}/{method_name}'
    os.makedirs(folder_path, exist_ok=True)

    with open(f'{folder_path}/system_message.txt', 'w') as f:
        f.write(res["system_message"])
    with open(f'{folder_path}/user_message.txt', 'w') as f:
        f.write(res["user_message"])
    with open(f'{folder_path}/response.txt', 'w') as f:
        f.write(res["response"])

df_output_file = output_folder + "/android_services_methods.parquet"
df_ums.to_parquet(df_output_file)
print(f"DataFrame serialized and saved to {df_output_file}")

/tmp/ipykernel_25050/4015053165.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_ums['json_answer'] = None
Processing rows:   1%|          | 1/85 [00:44<1:02:20, 44.53s/it]

Error extracting JSON from response: Error decoding JSON: Expecting value: line 1 column 1 (char 0)
getUserInfo
Lcom.android.server.pm.UserManagerService


Processing rows:  13%|█▎        | 11/85 [05:56<38:03, 30.86s/it] 

Error extracting JSON from response: Error decoding JSON: Expecting ',' delimiter: line 5 column 35 (char 119)
getSeedAccountOptions
Lcom.android.server.pm.UserManagerService


Processing rows:  99%|█████████▉| 84/85 [43:39<00:22, 22.87s/it]

Error extracting JSON from response: Error decoding JSON: Expecting value: line 2 column 1 (char 1)
getUserHandle
Lcom.android.server.pm.UserManagerService


Processing rows: 100%|██████████| 85/85 [44:09<00:00, 31.17s/it]

DataFrame serialized and saved to ./output/week1/output1/android_services_methods.parquet


In [101]:
df_ums

,service_name,class,method,depths,access control level,json_answer
325,Lcom.android.server.pm.UserManagerService,Lcom/android/server/pm/UserManagerService,getUserInfo(I)Landroid/content/pm/UserInfo;,"[{'depth': 0, 'trace': 'invokestatic < Applica...",SYS_OR_SIG,{'Sinks': [['int var5 = UserHandle.getCallingU...
326,Lcom.android.server.pm.UserManagerService,Lcom/android/server/pm/UserManagerService,getUserBadgeResId(I)I,"[{'depth': 0, 'trace': 'invokevirtual < Applic...",SYS_OR_SIG,{'Sinks': [['int var5 = UserHandle.getCallingU...
327,Lcom.android.server.pm.UserManagerService,Lcom/android/server/pm/UserManagerService,isUserSwitcherEnabled(I)Z,"[{'depth': 0, 'trace': '3 = getfield < Applica...",SYS_OR_SIG,{'Sinks': [['int var5 = UserHandle.getCallingU...
328,Lcom.android.server.pm.UserManagerService,Lcom/android/server/pm/UserManagerService,getProfiles(IZ)Ljava/util/List;,"[{'depth': 0, 'trace': '8 = new <Application,L...",SYS_OR_SIG,{'Sinks': [['int var5 = UserHandle.getCallingU...
329,Lcom.android.server.pm.UserManagerService,Lcom/android/server/pm/UserManagerService,getUserRestrictionSources(Ljava/lang/String;I)...,"[{'depth': 0, 'trace': '9 = new <Application,L...",SYS_OR_SIG,{'Sinks': [['int var5 = UserHandle.getCallingU...
...,...,...,...,...,...,...
405,Lcom.android.server.pm.UserManagerService,Lcom/android/server/pm/UserManagerService,clearSeedAccountData(I)V,"[{'depth': 0, 'trace': 'invokestatic < Applica...",SYS_OR_SIG,{'Sinks': [['int var5 = UserHandle.getCallingU...
406,Lcom.android.server.pm.UserManagerService,Lcom/android/server/pm/UserManagerService,getProfileType(I)Ljava/lang/String;,"[{'depth': 0, 'trace': 'invokevirtual < Applic...",SYS_OR_SIG,{'Sinks': [['int var5 = UserHandle.getCallingU...
407,Lcom.android.server.pm.UserManagerService,Lcom/android/server/pm/UserManagerService,createProfileForUserEvenWhenDisallowedWithThro...,"[{'depth': 0, 'trace': 'invokestatic < Applica...",SYS_OR_SIG,{'Sinks': [['int var5 = UserHandle.getCallingU...
408,Lcom.android.server.pm.UserManagerService,Lcom/android/server/pm/UserManagerService,getUserHandle(I)I,"[{'depth': 0, 'trace': '3 = getfield < Applica...",NONE,{'Sinks': [['int var5 = UserHandle.getCallingU...
